## Introduction to `panel` and `hvplot` with Python

In this assignment, you will be introduced to two Python libraries, `panel` and `hvplot`, which aid in visualizing data in an elegant and interactive way. For now, we will focus on the basics without introducing interactivity.

### Panel
Panel is a Python library for creating interactive web apps, allowing you to easily arrange visual components (layout) and make them respond to user input (reactivity).

<img src="images/panel_logo.png" width="200">

### hvPlot
HvPlot offers an intuitive interface to visualize your data interactively. The plots get created in bokeh by default.


<img src="images/hvplot_logo.svg" width="800">




**Context:** We have a dataset named "df" that consists of the following columns:
1. `date`: Represents the date of a school shooting.
2. `prevalence_ss`: Number of tweets on that day about the school shooting.
3. `number_victims`: Number of victims in the school shooting on that date.
4. `state`: The state in the USA where the shooting occurred.

We aim to study and visualize the correlation between the number of tweets and the number of victims on school shootings on any given day.


In [2]:
import pandas as pd
import panel as pn
import hvplot.pandas
import hvplot
import numpy as np
pn.extension()

# change the paths on google collab
path = "data/tweets_and_ss.tsv"
#path = "https://github.com/jgarciab/workshop_data_viz/raw/refs/heads/main/advanced_2024/data/tweets_and_ss.tsv"

df = pd.read_csv(path, sep="\t", parse_dates=["date"])
df.head()

,date,prevalence_ss,number_victims,state
0,2008-09-09,7.955449,NaN,NaN
1,2008-09-10,7.704160,NaN,NaN
2,2008-09-11,14.102564,NaN,NaN
3,2008-09-12,7.581501,NaN,NaN
4,2008-09-13,6.385696,NaN,NaN



### Part 1: Introduction

**hvplot:** An extension of HoloViews, `hvplot` provides a high-level API to visualize data with the possibility of creating interactive plots using Bokeh. 

**panel:** Panel is a high-level app and dashboarding solution for Python. It allows you to create interactive web apps and dashboards using tools you are already familiar with.


In [3]:
# Plotting number_of_tweets vs number_victims
plot = df.hvplot.scatter(x='prevalence_ss', 
                         y='number_victims', 
                         by='state', 
                         title='Tweets vs. Victims'
                        )

plot

:NdOverlay   [state]
   :Scatter   [prevalence_ss]   (number_victims)

### Exercise 1

Run `hvplot.help("scatter")` and change the options of the plot, especially labels, axis (made them logarithmic)

### Exercise 2

Create a bar plot to show the total number of victims by state, and a histogram showing the log(prevalence_ss).
`hvPlot` does not aggregate the data for you. You'll have to aggregate it yourself e.g. `df.groupby("state")[["number_victims"]].sum()`

### Exercise 3

Create a line plot of `number_of_tweets`  (prevalence_ss) over `date`.
Overlay a scatter plot of `number_victims` over `date`.

### Exercise 4

Objective: Use panel to organize visualizations and text into a coherent layout.

Task: Using the last plot, create a layout in panel that arranges:
- A Markdown pane on the left, which provides a brief description or analysis of the data.
- The plot on the right
    
End with `layout.servable()` or `layout.show()`
- `.servable()` is a way to explicitly specify which part of your code you intend to be the main app or visualization when serving with Panel. It doesn't have an immediate effect in Jupyter notebook, but still marks the object as servable.
- `.show()` launches a server and open the app in a new tab



# Solutions

### Exercise 2

In [13]:
# Potential solution

# Aggregating data for bar plot
state_victims_agg = df.groupby("state")["number_victims"].sum().sort_values()

# Bar plot for total number of victims by state
bar_plot = state_victims_agg.hvplot.barh(
    y="number_victims", 
    title="Total Number of Victims by State", 
    ylabel="State", 
    xlabel="Total Victims"
)


# Histogram of the log of prevalence_ss
df['log_prevalence_ss'] = np.log10(df['prevalence_ss'])
hist_plot = df.hvplot.hist(
    y='log_prevalence_ss', 
    bins=40,
    bin_range=(0,4),
    title="Histogram of log(Prevalence of Tweets)",
    xlabel="log(Prevalence of Tweets)", 
    ylabel="Frequency"
)

pn.Column(bar_plot, hist_plot)


/Users/garci061/miniforge3/envs/test3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Column
    [0] HoloViews(Bars, height=300, sizing_mode='fixed', width=700)
    [1] HoloViews(Histogram, height=300, sizing_mode='fixed', width=700)

### Exercise 3

In [14]:
# Number of tweets over time
line_plot = df.hvplot.line(
    x="date",
    y="prevalence_ss", 
    xlabel="Time", 
    ylabel="Tweets (blue) or Victims (red)",
    logy=True,
    ylim=(0.8, None)
)

# Number of tweets over time
scatter_plot = df.hvplot.scatter(
    x="date",
    y="number_victims", 
    color="tomato"
)


line_plot * scatter_plot


:Overlay
   .Curve.I   :Curve   [date]   (prevalence_ss)
   .Scatter.I :Scatter   [date]   (number_victims)

### Exercise 4

In [15]:
# Aggregating data for bar plot and sorting by total number of victims
state_victims_agg = df.groupby("state")[["number_victims"]].sum().sort_values(by="number_victims", ascending=False)


# Markdown description
description = """
# School Shootings Data Analysis

This dashboard provides a brief analysis of school shootings in the United States based on Twitter data.

- **Total Number of Victims by State**: The bar chart shows the total number of victims in each state. States with the highest number of victims are listed first.
- **Tweet Prevalence**: The histogram illustrates the distribution of the log-transformed prevalence of tweets about school shootings. Most tweet prevalence values are concentrated around lower log values.

These visualizations can help highlight trends and patterns in both public discourse and the impact of these events.
"""

# Create the layout with Markdown on the left and plots on the right
layout = pn.Row(
    pn.Column(pn.pane.Markdown(description, width=300)),
    pn.Column(bar_plot, hist_plot)
)

# Display the layout
layout.servable()


Row
    [0] Column
        [0] Markdown(str, width=300)
    [1] Column
        [0] HoloViews(Bars, height=300, sizing_mode='fixed', width=700)
        [1] HoloViews(Histogram, height=300, sizing_mode='fixed', width=700)